<a href="https://colab.research.google.com/github/Harshi07-c/Enhancing-LLM-s-Response-for-Lesson-preparation-Material-by-Prompt-Refinement-Technique/blob/main/content__eval__RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf

In [ ]:
from IPython.display import display
from IPython.display import Markdown
import textwrap

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [ ]:
import google.generativeai as genai
from google.colab import userdata

In [ ]:
import os
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)


In [ ]:
model = genai.GenerativeModel(model_name = "gemini-pro")
model

genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
)

In [ ]:
response = model.generate_content("What is an Array?")

In [ ]:
to_markdown(response.text)

> An array is a data structure that stores a collection of elements of the same type. Each element in the array is identified by an index, which is a unique number. Arrays are often used to store data that is related in some way, such as a list of names, ages, or grades.
> 
> Arrays can be one-dimensional or multi-dimensional. A one-dimensional array is a simple list of elements, while a multi-dimensional array is an array of arrays. For example, a two-dimensional array could be used to store a table of data, with each row and column representing a different value.
> 
> Arrays are an important data structure because they provide a convenient way to store and access data. They are also relatively easy to implement, which makes them a good choice for many different applications.

In [ ]:
import json
from time import sleep

In [ ]:
with open("/content/test_dataset.json",'r') as file :
  data = json.load(file)

In [ ]:
data

[{'query': 'Compare and contrast different sorting techniques such as bubble sort, selection sort, and insertion sort, highlighting their time and space complexities.',
  'ZS': 'Lecture Preparation Material:\n\nTitle: Sorting Techniques: Bubble Sort, Selection Sort, and Insertion Sort\n\nIntroduction:\nSorting is a fundamental operation in computer science that involves arranging elements in a specific order. There are various sorting techniques available, each with its own advantages and disadvantages. In this lecture, we will compare and contrast three popular sorting techniques: bubble sort, selection sort, and insertion sort. We will discuss their algorithms, time complexity, and space complexity.\n\n1. Bubble Sort:\n- Bubble sort is a simple sorting algorithm that repeatedly steps through the list, compares adjacent elements, and swaps them if they are in the wrong order.\n- Time Complexity: The worst-case and average-case time complexity of bubble sort is O(n^2), where n is the n

#Implementing Gemini API using langchain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY)

In [ ]:
result = llm.invoke("What is an Array?")

In [ ]:
to_markdown(result.content)

> An array is a collection of elements, all of the same type, that are referenced by a common name and an index. Each element in an array is assigned a unique index, which is used to access the element. Arrays are often used to store data that is related in some way, such as the names of students in a class or the grades of students on a test.
> 
> Arrays can be one-dimensional, two-dimensional, or even higher-dimensional. A one-dimensional array is a simple list of elements, while a two-dimensional array is a table of elements. Higher-dimensional arrays are less common, but they can be used to represent complex data structures, such as matrices or graphs.
> 
> Arrays are a powerful data structure that can be used to store and organize data in a variety of ways. They are relatively easy to use, and they can be accessed quickly and efficiently.

#Implementing RAG

In [ ]:
!sudo apt -y -qq install tesseract-ocr libtesseract-dev

!sudo apt-get -y -qq install poppler-utils libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

!pip install langchain


The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 121752 files and directories

In [ ]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY,
                             temperature=0.2,convert_system_message_to_human=True)

In [ ]:
pdf_loader = PyPDFLoader("/content/combinepdf.pdf")
pages = pdf_loader.load_and_split()
print(pages[3].page_content)

Lecture 1 - Introduction to Design and analysis of  algorithms 
 
What is an algorithm?        
 Algorithm is a set of steps to complete a task.     
 For example,            
 Task: to make a cup of tea.        
 Algorithm:  
• add water and milk to the kettle,  
• boilit, add tea leaves, 
• Add sugar, and then serve it in cup. 
What is Computer algorithm ? 
‘’a set of steps to accomplish or complete a task t hat is described precisely enough that a 
computer can run it’’. 
Described precisely : very difficult for a machine to know how much wat er, milk to be added 
etc. in the above tea making algorithm. 
These algorithmsrun on computers or computational d evices.Forexample, GPS in our 
smartphones, Google hangouts. 
GPS uses shortest path algorithm. Online shopping uses cryptography which uses RSA 
algorithm.  
Characteristics of an algorithm:- 
• Must take an input. 
• Must give some output(yes/no,valueetc.) 
• Definiteness –each instruction is clear and unambig uous. 
• Finiteness

In [ ]:
len(pages)

297

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)

In [ ]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True

)

In [ ]:
zs ='Lecture Preparation Material:\n\nTitle: Sorting Techniques: Bubble Sort, Selection Sort, and Insertion Sort\n\nIntroduction:\nSorting is a fundamental operation in computer science that involves arranging elements in a specific order. There are various sorting techniques available, each with its own advantages and disadvantages. In this lecture, we will compare and contrast three popular sorting techniques: bubble sort, selection sort, and insertion sort. We will discuss their algorithms, time complexity, and space complexity.\n\n1. Bubble Sort:\n- Bubble sort is a simple sorting algorithm that repeatedly steps through the list, compares adjacent elements, and swaps them if they are in the wrong order.\n- Time Complexity: The worst-case and average-case time complexity of bubble sort is O(n^2), where n is the number of elements in the list. The best-case time complexity is O(n) when the list is already sorted.\n- Space Complexity: Bubble sort has a space complexity of O(1) as it only requires a constant amount of extra space.\n\n2. Selection Sort:\n- Selection sort is another simple sorting algorithm that divides the input list into two parts: a sorted sublist and an unsorted sublist. It repeatedly selects the smallest element from the unsorted sublist and swaps it with the first unsorted element.\n- Time Complexity: The time complexity of selection sort is O(n^2) in all cases, as it always requires n^2 comparisons.\n- Space Complexity: Selection sort has a space complexity of O(1) as it only requires a constant amount of extra space.\n\n3. Insertion Sort:\n- Insertion sort is a simple sorting algorithm that builds the final sorted list one element at a time. It iterates through the list, removing one element at a time and inserting it into its correct position in the sorted sublist.\n- Time Complexity: The worst-case and average-case time complexity of insertion sort is O(n^2), where n is the number of elements in the list. The best-case time complexity is O(n) when the list is already sorted.\n- Space Complexity: Insertion sort has a space complexity of O(1) as it only requires a constant amount of extra space.\n\nComparison:\n- Bubble sort, selection sort, and insertion sort are all comparison-based sorting algorithms with quadratic time complexity.\n- Bubble sort and insertion sort are stable sorting algorithms, meaning that the relative order of equal elements is preserved. Selection sort is not stable.\n- Bubble sort and insertion sort are in-place sorting algorithms, meaning that they do not require additional space for sorting. Selection sort is also in-place.\n\nConclusion:\nIn conclusion, bubble sort, selection sort, and insertion sort are simple sorting techniques with quadratic time complexity. While they may not be the most efficient sorting algorithms for large datasets, they are easy to implement and understand. Understanding the algorithms, time complexity, and space complexity of these sorting techniques is essential for choosing the right sorting algorithm for a given problem.'
fs = 'Topic: Sorting Techniques - Bubble Sort, Selection Sort, Insertion Sort\n\n2 - Objectives:\n- Understand the concept of sorting techniques in computer science.\n- Learn about the different sorting algorithms such as bubble sort, selection sort, and insertion sort.\n- Analyze the time and space complexities of each sorting technique.\n- Implement and compare the performance of different sorting algorithms.\n- Explore real-life applications of sorting algorithms.\n\n3 - Theory:\nSorting techniques are essential algorithms used to arrange elements in a specific order. Different sorting algorithms have varying time and space complexities, making them suitable for different scenarios. In this lecture, we will focus on three popular sorting techniques: bubble sort, selection sort, and insertion sort.\n\n- Bubble Sort: Bubble sort is a simple sorting algorithm that repeatedly steps through the list, compares adjacent elements, and swaps them if they are in the wrong order. It has a time complexity of O(n^2) in the worst-case scenario and a space complexity of O(1).\n\n- Selection Sort: Selection sort is another simple sorting algorithm that divides the input list into two parts: a sorted sublist and an unsorted sublist. It repeatedly selects the smallest element from the unsorted sublist and swaps it with the first unsorted element. Selection sort has a time complexity of O(n^2) and a space complexity of O(1).\n\n- Insertion Sort: Insertion sort is a sorting algorithm that builds the final sorted list one element at a time. It takes each element from the input list and inserts it into its correct position in the sorted list. Insertion sort has a time complexity of O(n^2) in the worst-case scenario and a space complexity of O(1).\n\n4 - Code Implementation:\nHere is an example of code implementation for bubble sort, selection sort, and insertion sort in Python:\n\n```python\n# Bubble Sort\ndef bubble_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if arr[j] > arr[j+1]:\n                arr[j], arr[j+1] = arr[j+1], arr[j]\n\n# Selection Sort\ndef selection_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        min_idx = i\n        for j in range(i+1, n):\n            if arr[j] < arr[min_idx]:\n                min_idx = j\n        arr[i], arr[min_idx] = arr[min_idx], arr[i]\n\n# Insertion Sort\ndef insertion_sort(arr):\n    n = len(arr)\n    for i in range(1, n):\n        key = arr[i]\n        j = i-1\n        while j >= 0 and key < arr[j]:\n            arr[j+1] = arr[j]\n            j -= 1\n        arr[j+1] = key\n\n# Example usage\narr = [64, 34, 25, 12, 22, 11, 90]\nbubble_sort(arr)\nprint("Bubble Sort:", arr)\n\narr = [64, 34, 25, 12, 22, 11, 90]\nselection_sort(arr)\nprint("Selection Sort:", arr)\n\narr = [64, 34, 25, 12, 22, 11, 90]\ninsertion_sort(arr)\nprint("Insertion Sort:", arr)\n```\n\n5 - Explanation of Code and Complexity:\nThe code above demonstrates the implementation of bubble sort, selection sort, and insertion sort algorithms in Python. Each sorting technique is applied to the same input array, and the sorted array is printed.\n\n- Bubble Sort: The time complexity of bubble sort is O(n^2), and the space complexity is O(1).\n- Selection Sort: The time complexity of selection sort is O(n^2), and the space complexity is O(1).\n- Insertion Sort: The time complexity of insertion sort is O(n^2), and the space complexity is O(1).\n\n6 - Real-Life Applications:\nSorting techniques are used in various real-life scenarios, such as:\n- Sorting and organizing large datasets in databases.\n- Optimizing search algorithms by maintaining sorted data.\n- Implementing scheduling algorithms in operating systems.\n- Arranging elements in graphical user interfaces for better user experience.\n- Analyzing and processing financial data efficiently.\n\n7 - Conclusion:\nIn this lecture, we discussed three popular sorting techniques: bubble sort, selection sort, and insertion sort. We explored their implementations, time and space complexities, and real-life applications. Understanding different sorting algorithms is crucial for efficient data organization and problem-solving in computer science.\n\n8 - Quiz:\n1. What is the time complexity of bubble sort?\n  a) O(n)\n  b) O(n log n)\n  c) O(n^2)\n\n2. Which sorting technique builds the final sorted list one element at a time?\n  a) Bubble Sort\n  b) Selection Sort\n  c) Insertion Sort\n\n3. What is the space complexity of selection sort?\n  a) O(1)\n  b) O(log n)\n  c) O(n)\n\nAnswers: 1-c, 2-c, 3-a\n\n9 - References:\n- "Introduction to Algorithms" by Thomas H. Cormen, Charles E. Leiserson, Ronald L. Rivest, and Clifford Stein\n- GeeksforGeeks website: https://www.geeksforgeeks.org/\n- Khan Academy: https://www.khanacademy.org/computing/computer-science/algorithms/sorting-searching'
tb = '1 - Extract the Topic from query delimited by triple backticks.\nSorting Techniques: Bubble Sort, Selection Sort, Insertion Sort\n\n2 - Write the objectives for the lecture.\n- Understand the concept of sorting algorithms.\n- Compare and contrast bubble sort, selection sort, and insertion sort.\n- Analyze the time and space complexities of each sorting technique.\n\n3 - Provide elaborate theory of the topic.\nSorting algorithms are used to arrange elements in a specific order. Bubble sort is a simple sorting algorithm that repeatedly steps through the list, compares adjacent elements, and swaps them if they are in the wrong order. Selection sort works by repeatedly finding the minimum element from the unsorted part of the list and swapping it with the first unsorted element. Insertion sort builds the final sorted array one element at a time by taking elements from the unsorted part and inserting them into their correct position in the sorted part.\n\n4 - Show code implementation of the topic as asked in the query.\n```python\n# Bubble Sort\ndef bubble_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if arr[j] > arr[j+1]:\n                arr[j], arr[j+1] = arr[j+1], arr[j]\n    return arr\n\n# Selection Sort\ndef selection_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        min_idx = i\n        for j in range(i+1, n):\n            if arr[j] < arr[min_idx]:\n                min_idx = j\n        arr[i], arr[min_idx] = arr[min_idx], arr[i]\n    return arr\n\n# Insertion Sort\ndef insertion_sort(arr):\n    n = len(arr)\n    for i in range(1, n):\n        key = arr[i]\n        j = i - 1\n        while j >= 0 and key < arr[j]:\n            arr[j + 1] = arr[j]\n            j -= 1\n        arr[j + 1] = key\n    return arr\n\narr = [64, 34, 25, 12, 22, 11, 90]\nprint("Bubble Sort:", bubble_sort(arr.copy()))\nprint("Selection Sort:", selection_sort(arr.copy()))\nprint("Insertion Sort:", insertion_sort(arr.copy()))\n```\n\n5 - Explain the code you wrote before with respect to the topic elaborately.\nThe code above demonstrates the implementation of bubble sort, selection sort, and insertion sort algorithms in Python. Each sorting technique is applied to a sample array to showcase how they rearrange the elements in ascending order. Bubble sort compares adjacent elements and swaps them if they are in the wrong order. Selection sort finds the minimum element and swaps it with the first unsorted element. Insertion sort inserts elements into their correct position in the sorted part of the array.\n\n6 - Explain its time and space complexity of the code you wrote before.\n- Bubble Sort:\n  Time Complexity: O(n^2) in the worst and average case, O(n) in the best case.\n  Space Complexity: O(1)\n- Selection Sort:\n  Time Complexity: O(n^2) in all cases.\n  Space Complexity: O(1)\n- Insertion Sort:\n  Time Complexity: O(n^2) in the worst case, O(n) in the best case.\n  Space Complexity: O(1)\n\n7 - Provide the applications of topic elaborately.\n- Bubble Sort: It is rarely used in practice due to its inefficiency for large datasets.\n- Selection Sort: It is simple and easy to implement, making it suitable for small datasets.\n- Insertion Sort: It is efficient for small datasets and is often used in practice for sorting small arrays.\n\n8 - Provide a conclusion in which the summary of whatever has been discussed is present.\nIn conclusion, bubble sort, selection sort, and insertion sort are basic sorting algorithms with different time and space complexities. Bubble sort is inefficient for large datasets, while selection sort and insertion sort are more suitable for small datasets. Understanding the differences between these sorting techniques is essential for choosing the appropriate algorithm based on the size of the dataset.\n\n9 - Provide a short quiz of at most 3 questions on the lesson with answers\n1. What is the time complexity of bubble sort in the worst case?\n   A) O(n)\n   B) O(n^2)\n   C) O(log n)\n   D) O(n!)\n   Answer: B) O(n^2)\n\n2. Which sorting algorithm repeatedly finds the minimum element and swaps it with the first unsorted element?\n   A) Bubble Sort\n   B) Selection Sort\n   C) Insertion Sort\n   D) Merge Sort\n   Answer: B) Selection Sort\n\n3. When is insertion sort most efficient compared to bubble sort and selection sort?\n   A) For large datasets\n   B) For unsorted datasets\n   C) For small datasets\n   D) For sorted datasets\n   Answer: C) For small datasets\n\n10 - Provide references with proper links to study the topics further.\n- Bubble Sort: https://en.wikipedia.org/wiki/Bubble_sort\n- Selection Sort: https://en.wikipedia.org/wiki/Selection_sort\n- Insertion Sort: https://en.wikipedia.org/wiki/Insertion_sort'

In [ ]:
query = "Compare and contrast different sorting techniques such as bubble sort, selection sort, and insertion sort, highlighting their time and space complexities."

In [ ]:
question = query
result = qa_chain({"query": question})
Markdown(result["result"])

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


**Bubble Sort**

* **Time Complexity:** O(n^2)
* **Space Complexity:** O(1)
* **Description:** Bubble sort repeatedly compares adjacent elements and swaps them if they are in the wrong order. It continues to iterate through the list until no more swaps are needed.

**Selection Sort**

* **Time Complexity:** O(n^2)
* **Space Complexity:** O(1)
* **Description:** Selection sort finds the minimum element from the unsorted portion of the list and swaps it with the leftmost unsorted element. This process is repeated until the entire list is sorted.

**Insertion Sort**

* **Time Complexity:** O(n^2)
* **Space Complexity:** O(1)
* **Description:** Insertion sort builds the sorted list one element at a time by inserting each unsorted element into its correct position in the sorted portion of the list.

**Comparison**

| Feature | Bubble Sort | Selection Sort | Insertion Sort |
|---|---|---|---|
| Time Complexity | O(n^2) | O(n^2) | O(n^2) |
| Space Complexity | O(1) | O(1) | O(1) |
| Stability | Yes | No | Yes |
| Efficiency | Inefficient for large datasets | Inefficient for large datasets | More efficient than bubble and selection sort |

**Stability** refers to whether the algorithm maintains the relative order of equal elements in the input. Bubble sort and insertion sort are stable, while selection sort is not.

**Overall**

Bubble sort, selection sort, and insertion sort are simple sorting algorithms with O(n^2) time complexity. They are not suitable for large datasets, but they can be useful for small lists or for educational purposes.

In [ ]:
result['source_documents']

[Document(page_content='and in the average case it is half that, namely n(n−1)/4. Thus average and worst case\nnumber of steps for of Insertion Sort are both proportional to n2, and hence the average and\nworst case time complexities are both O(n2).\n9.6 Selection Sort\nSelection Sort is (not surprisingly) a form of selection sorting . It ﬁrst ﬁnds the smallest item\nand puts it into a[0] by exchanging it with whichever item is in that position already. Then\nit ﬁnds the second-smallest item and exchanges it with the item in a[1] . It continues this\nway until the whole array is sorted. More generally, at the ith stage, Selection Sort ﬁnds the\nith-smallest item and swaps it with the item in a[i-1] . Obviously there is no need to check\nfor theith-smallest item in the ﬁrst i−1 elements of the array.\nFor the example starting array 4 1 3 2 , Selection Sort ﬁrst ﬁnds the smallest item in\nthe whole array, which is a[1]=1 , and swaps this value with that in a[0] giving 1 4 3 2 .\nThen, fo

In [ ]:
template = f""" You are a professor who teaches computer science to students.You need to rank
lecture preparation materials for a query delimited by triple backticks.The material is generated by 3 different prompt
techniques called ZS,FS and TB.Ranking should be on the basis of the quality of the content under each heading .For you to judge
the quality, I am providing you with a context ,delimited by triple backticks, related to the query.Using this context, judge the materials
and the material which has the best content among  the three should be rank 1 ,followed by the next best material. If you think that 2 materials  similar in terms
of content quality , give them the same rank.
The lecture material are :
'ZS': {zs}
'FS': {fs}
'TB': {tb}
Now the output should look like =  TB : the rank you gave to tb, FS: the rank you gave to fs, ZS: the rank you gave to tb
The ranking should not be similar to the order in which the materials are given to you. Analyze the material on the basis of
the criteria given using the context provided to you and rank them accordingly.
  Query:
      ```{query}```
  Context : ```{result}```
  """

In [ ]:
ranking = llm.invoke(template)

In [ ]:
ranking.content

'TB: 1, FS: 3, ZS: 2'

In [ ]:
# Calculate the size of each subset
subset_size = len(data) // 10

# Create a dictionary to store the subsets
subset_dict = {}

# Create 10 subsets and store them in the dictionary
for i in range(10):
    subset_name = f'subset_{i+1}'
    subset_start = i * subset_size
    subset_end = (i + 1) * subset_size
    subset_dict[subset_name] = data[subset_start:subset_end]

# Print the names and lengths of the subsets
for subset_name, subset_data in subset_dict.items():
    print(f"{subset_name}: {len(subset_data)}")


subset_1: 5
subset_2: 5
subset_3: 5
subset_4: 5
subset_5: 5
subset_6: 5
subset_7: 5
subset_8: 5
subset_9: 5
subset_10: 5


In [ ]:
def rank_lecture_materials(query, zs_material, fs_material, tb_material):
    context = qa_chain({"query": query})
    # Rank lecture materials for the current query
    template = f""" You are a professor who teaches computer science to students.You need to rank
    lecture preparation materials for a query delimited by triple backticks.The material is generated by 3 different prompt
    techniques called ZS,FS and TB.Ranking should be on the basis of the quality of the content under each heading .For you to judge
    the quality, I am providing you with a context ,delimited by triple backticks, related to the query.Using this context, judge the materials
    and check if they have the coorect and elaborate content respective to the headings.The material which has the best content among  the three should
    be rank 1 ,followed by the next best material.If you think that 2 materials  similar in terms
    of content quality , give them the same rank.
    The lecture material are :
    'ZS': {zs_material}
    'FS': {fs_material}
    'TB': {tb_material}
    The output should look like = TB : the rank you gave to tb, FS: the rank you gave to fs, ZS: the rank you gave to zs .
    Don't give any reasonings as to why you gave a particular reasoning.There should only be rankings and no other text in the output.
    The ranking should be independent of the order in which the materials are given to you. Analyze the material on the basis of
    the criteria given using the context provided to you and rank them accordingly.
      Query:
          ```{query}```
      Context : ```{context}```
      """

    # Extract rankings from response
    rankings = llm.invoke(template).content

    return rankings



In [ ]:
final_ranks1 = []
final_ranks2 = []
final_ranks3 = []
final_ranks3 = []
final_ranks4 = []
final_ranks5 = []
final_ranks6 = []
final_ranks7 = []
final_ranks8 = []
final_ranks9 = []
final_ranks10 = []

In [ ]:
for entry in subset_dict['subset_10'] :
  query = entry['query']
  zs_material = entry['ZS']
  fs_material = entry['FS']
  tb_material = entry['TB']

  # Rank lecture materials for the current query
  query_rankings = rank_lecture_materials(query, zs_material, fs_material, tb_material)
  if query_rankings is not None:
    # Append rankings to the list
    final_ranks10.append(query_rankings)

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_hu

In [ ]:
final_ranks10

['TB : 1, FS: 2, ZS: 3',
 ' ZS : the rank you gave to zs, FS: the rank you gave to fs, TB: the rank you gave to tb\nZS : 3, FS: 2, TB: 1',
 'TB : 3, FS: 2, ZS: 1',
 ' TB : 2, FS: 1, ZS: 3',
 'FS: 2, TB: 3, ZS: 1']

In [ ]:
print(final_ranks9)
print(final_ranks8)
print(final_ranks7)
print(final_ranks6)
print(final_ranks5)
print(final_ranks4)
print(final_ranks3)
print(final_ranks2)
print(final_ranks1)


['TB : 1, FS : 3, ZS : 2', 'TB : 2, FS : 1, ZS : 3']
[' TB : 3, FS : 1, ZS : 2', ' TB : 1, FS: 2, ZS: 3', 'TB : 1, FS : 2, ZS : 3', 'FS : 2, TB : 1, ZS : 3', 'ZS : 3, FS : 2, TB : 1']
[' ZS:bad\nFS:good\nTB:best', 'TB : 3, FS: 2, ZS: 1', 'TB: ZS,FS,TB', 'ZS : 2\nFS : 1\nTB : 3', 'TB : 1, FS : 2, ZS : 3']
['TB : 1, FS: 2, ZS: 3', 'TB : 1, FS: 2, ZS: 3', 'ZS : 1, FS : 2, TB : 3', 'TB : 2, FS: 1, ZS: 3', 'TB : 1, FS : 2, ZS : 3']
['TB : 1, FS : 2, ZS : 3', 'TB : 1, FS: 2, ZS: 3', 'TB : 2, FS: 1, ZS: 3', ' ZS: 1, FS: 2, TB: 3', 'ZS : 1\nFS : 2\nTB : 3']
['FS: 2\nZS: 1\nTB: 3', 'TB: 3, FS: 2, ZS: 1', 'TB : 1, FS: 2, ZS: 3', 'TB : 1, FS: 2, ZS: 3', 'TB : 2, FS : 1, ZS : 3']
['TB : 1, FS: 2, ZS: 3', 'TB : 2\nFS : 1\nZS : 3', 'ZS : 2, FS: 1, TB: 3', 'FS : rank 1, TB : rank 2, ZS : rank 3', ' TB : 2, FS: 1, ZS: 3']
['TB : 1\nFS: 2\nZS: 3', 'TB : 3, FS: 2, ZS: 1', 'TB : 1, FS: 2, ZS: 3', 'ZS : 1\nFS : 2\nTB : 3', 'TB : 1\nFS : 2\nZS : 3']
['ZS : 1, FS: 2, TB: 3', 'TB : 1, FS: 2, ZS: 3', 'TB : 2,

In [ ]:
print(final_ranks1)

['ZS : 1, FS: 2, TB: 3', 'TB : 1, FS: 2, ZS: 3', 'TB : 2, FS: 1, ZS: 3', 'TB : 1, FS: 2, ZS: 3', 'FS : 2, TB : 1, ZS : 3']
